In [2]:
# Selenium imports
!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.headless = True
!pip install webdriver-manager
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd
import regex as re
import requests
from bs4 import BeautifulSoup

C:\Users\Felipe\AppData\Local\Temp\ipykernel_17992\599045925.py:7: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  chrome_options.headless = True


In [3]:
# General Helpers

def price_multiple_replace(text, dict=None):
    """
    Multiple regex replacements (to replace in key, replacement in value) 
    """
    if not dict:
        dict = {"," : ".", "." : ""}
        
    # Create a regular expression  from the dictionary keys
    regex = re.compile("(%s)" % "|".join(map(re.escape, dict.keys())))
    
    # For each match, look-up corresponding value in dictionary
    return float(regex.sub(lambda mo: dict[mo.string[mo.start():mo.end()]], text))

# Argenprop

In [94]:
headers = {
    "Accept-Language": "en-US,en;q=0.5",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:40.0) Gecko/20100101 Firefox/40.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Referer": "http://thewebsite.com",
    "Connection": "keep-alive"}

url_argenprop = 'https://www.argenprop.com/departamento-y-casa-venta-y-alquiler-y-alquiler-temporal-pagina-'

cantidad_paginas_a_scrapear = 10

# for pag in range(cantidad_paginas_a_scrapear):

result_ap = requests.get(url_argenprop, headers=headers)

soup_ap = BeautifulSoup(result_ap.text, 'lxml')

In [95]:
all_columns_ap = []


# Links
base_link = 'https://www.argenprop.com'
div_container = soup_ap.find_all('div',{'class':'listing__item'})
links_ap = []
for div in div_container:
    a_link = div.find_all('a', href=True)
    last_link = a_link[0]['href']
    links_ap.append(base_link+last_link)
all_columns_ap.append(links_ap)


# Service
all_columns_ap.append(['argenprop']*len(links_ap))


# City
city_ap = soup_ap.find_all('p', {'class':'card__title--primary show-mobile'})
city_ap = [c.text for c in city_ap]
all_columns_ap.append(city_ap)

# Adress
address_ap = soup_ap.find_all('h2', {'class':'card__address'})
address_ap = [c.text.strip() for c in address_ap]
address_ap = [re.findall('.+', str(c))[0] for c in address_ap]
all_columns_ap.append(address_ap)

# Property Type
property_type_ap = soup_ap.find_all('p', {'class':'card__title--primary hide-mobile'})
property_type_ap = [c.text.split(' ')[0] for c in property_type_ap]
all_columns_ap.append(property_type_ap)

# Contract type
contract_type_ap = soup_ap.find_all('p', {'class':'card__title--primary hide-mobile'})
contract_type_ap = [c.text.split(' ')[2] for c in contract_type_ap]
all_columns_ap.append(contract_type_ap)

# Rooms
ul_container = soup_ap.find_all('ul', {'class':'card__main-features'})
rooms_ap = []
for ul in ul_container:
    amb = re.findall('(\d+)\sambiente', str(ul.find_all('span')))
    if len(amb)>0:
        rooms_ap.append(int(amb[0]))
    else:
        rooms_ap.append(1)
all_columns_ap.append(rooms_ap)


# Price and currency type
buy_rent_currency_ap = []
buy_rent_price_ap = []

expensas_currency_ap = []
expensas_price_ap = []

all_currency_and_price_ap = soup_ap.find_all('p', {'class':'card__price'})
all_currency_and_price_ap = [c.text for c in all_currency_and_price_ap]
all_currency_and_price_ap = [re.findall('(USD|\$)\s?(\d+(?:[,.]\d+)*)', str(c)) for c in all_currency_and_price_ap]

for cp in all_currency_and_price_ap:
    if len(cp)>0:
        buy_rent_currency_ap.append(cp[0][0])
        buy_rent_price_ap.append(price_multiple_replace(cp[0][1]))
        if len(cp)>1:
            expensas_currency_ap.append(cp[1][0])
            expensas_price_ap.append(price_multiple_replace(cp[1][1]))
        else:
            expensas_currency_ap.append('consultar_precio')
            expensas_price_ap.append(0)
    else:
        buy_rent_currency_ap.append('consultar_precio')
        buy_rent_price_ap.append(0)
        expensas_currency_ap.append('consultar_precio')
        expensas_price_ap.append(0)
        
all_columns_ap.append(buy_rent_currency_ap)
all_columns_ap.append(expensas_currency_ap)
all_columns_ap.append(buy_rent_price_ap)
all_columns_ap.append(expensas_price_ap)

# Mercado Libre

In [89]:
# Helpers
def get_expensas_price(links):
    
    """
    For Mercado Libre
    """
    expensas_price = []

    link_expensas = [l for l in links if 'departamento' in l]

    for l_expensas in link_expensas:
        result = requests.get(l_expensas, headers=headers)
        soup = BeautifulSoup(result.text, 'lxml')
        try:
            exp = soup.find('p', {'class':'ui-pdp-color--GRAY ui-pdp-size--XSMALL ui-pdp-family--REGULAR ui-pdp-maintenance-fee-ltr'}).text
        except:
            exp = '0'
        expensas_price.append(exp)

    expensas_price = [re.findall('\d+(?:[,.]\d+)*', ex)[0] for ex in expensas_price]
    expensas_price = [int(price_multiple_replace(c)) for c in expensas_price]
    
    return expensas_price

In [90]:
headers = {
    "Accept-Language": "en-US,en;q=0.5",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:40.0) Gecko/20100101 Firefox/40.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Referer": "http://thewebsite.com",
    "Connection": "keep-alive"}

url_ml = 'https://inmuebles.mercadolibre.com.ar/departamentos/'


result_ml = requests.get(url_ml, headers=headers)

soup_ml = BeautifulSoup(result_ml.text, 'lxml')

In [91]:
all_columns_ml = []


# Links
div_container = soup_ml.find_all('div',{'class':'ui-search-result__wrapper shops__result-wrapper'})
links_ml = []
for div in div_container:
    a_link = div.find_all('a', href=True)
    link = a_link[0]['href']
    links_ml.append(link)
all_columns_ml.append(links_ml)

# Service
all_columns_ml.append(['mercado_libre']*len(links_ml))

# City
city_ml = soup_ml.find_all('span', {'class':'ui-search-item__group__element ui-search-item__location shops__items-group-details'})
city_ml = [c.text for c in city_ml]
all_columns_ml.append(city_ml)


# Adress
address_ml = soup_ml.find_all('span', {'class':'ui-search-item__group__element ui-search-item__location shops__items-group-details'})
address_ml = [c.text.split(',')[0] for c in address_ml]
all_columns_ml.append(address_ml)

# Property type
property_type_ml = soup_ml.find_all('span', {'class':'ui-search-item__group__element ui-search-item__subtitle shops__items-group-details'})
property_type_ml = [c.text.split(' en ')[0] for c in property_type_ml]
all_columns_ml.append(property_type_ml)

# Contract type
contract_type_ml = soup_ml.find_all('span', {'class':'ui-search-item__group__element ui-search-item__subtitle shops__items-group-details'})
contract_type_ml = [c.text.split(' en ')[-1] for c in contract_type_ml]
all_columns_ml.append(contract_type_ml)

# Rooms
rooms_ml = soup_ml.find_all('li', {'class':'ui-search-card-attributes__attribute'})
rooms_ml = [int(c.text.split(' ')[0]) for c in rooms_ml if 'm²' not in c.text]
all_columns_ml.append(rooms_ml)


# All Currency type
buy_rent_currency_ml = soup_ml.find_all('span', {'class':'price-tag-symbol'})
buy_rent_currency_ml = [c.text.replace('U$S', 'USD') for c in buy_rent_currency_ml]
buy_rent_currency_ml = buy_rent_currency_ml[len(buy_rent_currency_ml)-len(links_ml):] # Eliminate those with no link
expensas_currency_ml = ['$']*len(links_ml)
all_columns_ml.append(buy_rent_currency_ml)
all_columns_ml.append(expensas_currency_ml)


# Buy Rent Price
buy_rent_price_ml = []
buy_rent_price_ml = soup_ml.find_all('span', {'class':'price-tag-fraction'})
buy_rent_price_ml = [c.text for c in buy_rent_price_ml]
buy_rent_price_ml = [int(price_multiple_replace(c)) for c in buy_rent_price_ml]
buy_rent_price_ml = buy_rent_price_ml[len(buy_rent_price_ml)-len(links_ml):] # Eliminate those with no link
all_columns_ml.append(buy_rent_price_ml)


# Expensas Price
scrap_expensas_ML = True
if scrap_expensas_ML:
   expensas_price = get_expensas_price(links_ml)
else:
    expensas_price = [0]*len(links_ml)
all_columns_ml.append(expensas_price)

# Dataframe

In [96]:
all_columns = []

for i in range(len(all_columns_ml)):
    all_columns.append(all_columns_ml[i]+all_columns_ap[i])

In [110]:
col_names = ['links','service','adress','city','property_type','contract_type','rooms','buy_rent_currency','expensas_currency','buy_rent_price','expensas_price']
df = pd.DataFrame(all_columns).T
df.columns = col_names

# Zonaprop

In [122]:
headers = {
    "Accept-Language": "en-US,en;q=0.5",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:40.0) Gecko/20100101 Firefox/40.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Referer": "http://thewebsite.com",
    "Connection": "keep-alive"}

url_zp = 'https://www.zonaprop.com.ar/inmuebles-alquiler.html'


result_zp = requests.get(url_zp, headers=headers)

result_zp.status_code

403

In [123]:
url_zp = 'https://www.zonaprop.com.ar/inmuebles-alquiler.html'

In [124]:
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Downloading: 100%|██████████| 6.79M/6.79M [00:00<00:00, 52.9MB/s]
C:\Users\Felipe\AppData\Local\Temp\ipykernel_17992\1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [125]:
driver.get(url_zp)